# 使用RNN來做電影評論"情義分析",可以從評論得知對於該電影為"正評"或"負評"


### 初始準備

In [1]:
%env KERAS_BACKEND=tensorflow

env: KERAS_BACKEND=tensorflow


In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

### 讀入 IMDB 電影數據庫


In [3]:
from keras.datasets import imdb

Using TensorFlow backend.


In [4]:
(x_train0,y_train0),(x_test0,y_test0) = imdb.load_data(num_words = 5000)

### 資料除存以數字編碼

In [20]:
x_train0[24999]

[1,
 17,
 6,
 194,
 337,
 7,
 4,
 204,
 22,
 45,
 254,
 8,
 106,
 14,
 123,
 4,
 2,
 270,
 2,
 5,
 2,
 2,
 732,
 2098,
 101,
 405,
 39,
 14,
 1034,
 4,
 1310,
 9,
 115,
 50,
 305,
 12,
 47,
 4,
 168,
 5,
 235,
 7,
 38,
 111,
 699,
 102,
 7,
 4,
 4039,
 2,
 9,
 24,
 6,
 78,
 1099,
 17,
 2345,
 2,
 21,
 27,
 2,
 2,
 5,
 2,
 1603,
 92,
 1183,
 4,
 1310,
 7,
 4,
 204,
 42,
 97,
 90,
 35,
 221,
 109,
 29,
 127,
 27,
 118,
 8,
 97,
 12,
 157,
 21,
 2,
 2,
 9,
 6,
 66,
 78,
 1099,
 4,
 631,
 1191,
 5,
 2642,
 272,
 191,
 1070,
 6,
 2,
 8,
 2197,
 2,
 2,
 544,
 5,
 383,
 1271,
 848,
 1468,
 2,
 497,
 2,
 8,
 1597,
 2,
 2,
 21,
 60,
 27,
 239,
 9,
 43,
 2,
 209,
 405,
 10,
 10,
 12,
 764,
 40,
 4,
 248,
 20,
 12,
 16,
 5,
 174,
 1791,
 72,
 7,
 51,
 6,
 1739,
 22,
 4,
 204,
 131,
 9]

In [6]:
print("訓練資料長度 =",len(x_train0))
print("測試資料長度 =",len(x_test0))

訓練資料長度 = 25000
測試資料長度 = 25000


In [7]:
len(x_train0[24999])

153

In [8]:
len(x_train0[9982])

156

### 輸出資料部份

輸出方面應該很容易想像, 我們來看看前 10 筆。結果自然就是 0 (負評) 或 1 (正評)。

In [9]:
y_train0[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

### 送入神經網路的輸入處理

雖然 RNN 是可以處理不同長度的輸入, 在寫程式時我們還是要

* 設輸入文字長度的上限
* 把每段文字都弄成一樣長, 太短的後面補上 0

In [10]:
from keras.preprocessing import sequence

### 先將影評長度用成一樣(不一定需要,但較簡單)

In [11]:
x_train = sequence.pad_sequences(x_train0,maxlen = 150)
x_test = sequence.pad_sequences(x_test0,maxlen = 150)

In [12]:
y_train = y_train0
y_test = y_test0

### 使用LSTM

### 決定神經網路架構

* 先將 10000 維的文字壓到 N 維
* 然後用 K 個 LSTM 神經元做隱藏層
* 最後一個 output, 直接用 sigmoid 送出

### 建構我們的神經網路

文字我們用 1-hot 表示是很標準的方式, 不過要注意的是, 因為我們指定要 1 萬個字, 所以每個字是用 1 萬維的向量表示! 這一來很浪費記憶空間, 二來字和字間基本上是沒有關係的。我們可以用某種「合理」的方式, 把字壓到比較小的維度, 這些向量又代表某些意思 (比如說兩個字代表的向量角度小表相關程度大) 等等。

這聽來很複雜的事叫 "word embedding", 而事實上 Keras 會幫我們做。我們只需告訴 Keras 原來最大的數字是多少 (10000), 還有我們打算壓到幾維 (N)。

In [13]:
N = 3 #文字壓成3維
K1 = 20 #20神經元

In [14]:
from keras.models import Sequential
from keras.layers import Dense ,Embedding ,Dropout
from keras.layers import LSTM

In [15]:
model = Sequential()
model.add(Embedding(5000,N))
model.add(Dropout(0.35))
model.add(LSTM(K1))
model.add(Dense(100,activation = 'sigmoid'))
model.add(Dropout(0.2))

model.add(Dense(1,activation = 'sigmoid'))

### 組裝

這次我們用 binary_crossentropy 做我們的 loss function, 另外用一個很潮的 Adam 學習法。

In [16]:
model_info = model.compile(loss = 'binary_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 3)           15000     
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 3)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 20)                1920      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               2100      
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 19,121
Trainable params: 19,121
Non-trainable params: 0
_________________________________________________________________


## 訓練結果

In [18]:
model.fit(x_train,y_train,batch_size = 64,epochs = 4)

Epoch 1/4
25000/25000 [==============================] - 103s - loss: 0.5634 - acc: 0.6631   
Epoch 2/4
25000/25000 [==============================] - 104s - loss: 0.3307 - acc: 0.8605   
Epoch 3/4
25000/25000 [==============================] - 155s - loss: 0.2932 - acc: 0.8797   
Epoch 4/4
25000/25000 [==============================] - 156s - loss: 0.2717 - acc: 0.8908   
